Copy this file into the same folder as the data you want to analyse.

You can run a cell by hitting  `Shift + Enter` on the keyboard

A more extensive tutorial can be found [here](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook#gs.kwHTUSQ) Skip to: "Getting Started With Jupyter Notebooks" if you are hazy.

In [6]:
# Import and load some packages
%pylab notebook
import sfg2d
import os
# Set some global variables
vis_wl = 810
# Create some folders
if not os.path.isdir("figures"): 
    os.mkdir("figures")
if not os.path.isdir("combined"):
    os.mkdir("combined")

Populating the interactive namespace from numpy and matplotlib


# Backgroud

In [ ]:
# Fill in correct data path after the ./
# You can use the TAB key for autocompletion.
# Position your cursor after the ./ and hit the TAB key
bg0 = sfg2d.SfgRecord("./").make_avg().data # <- e.g. shot background scan
bg1 = sfg2d.SfgRecord("./").make_avg().data # <- e.g. long background scan

# Ir Profile

In [ ]:
q0 = sfg2d.SfgRecord("./") # Fill in path

q0 = q0.make_avg() # Calculates the frame wise average of a data set.
q0.base = bg0 # Choos correct baseline here. You can also just put a constan value.
q0.sub_base(inplace=True) # substracts baseline from data
q0.metadata["vis_wl"] = vis_wl # Sets a new vis_wl. Default is 810
q0._wavenumber = None # Reset Wavenumber so it gets recalculated using the above vis_wl
norm_spec = q0.data # <- This will be your dataset for normalization

In [ ]:
q0.plot(x_axis="wavenumber")
xlabel("Wavenumber 1/cm")
ylabel("Counts in {0}s".format(q0.metadata.get('exposure_time').seconds)
title("Quartz Normalization")
sfg2d.savefig("figures/norm") # Saves the figures as pixel and as vektor graphics.
q0.save("combined/quartz") # Saves intermediate result for easier reuse.

# Cross-Correlation

In [ ]:
cc0 = sfg2d.SfgRecord("./").make_avg() # Fill in path


cc0.base = bg0 # Use correct baseline
cc0.sub_base(inplace=True)
cc0.metadata["vis_wl"] = vis_wl
cc0._wavenumber = None

In [ ]:
fig, ax = subplots()
cc0.plot(y_pixel=slice(0,1), x_axis="wavenumber");
#xlim(4200, 5100) # Select some data range in x_axis coordinates
xlabel("Wavenumber in 1/cm")
ylabel("Counts in {0}s".format(cc0.metadata.get('exposure_time').seconds)
title("Pump-Probe IIV")
sfg2d.savefig("figures/cc_spectra")

fig, ax = subplots()
#x_slice = slice(363, 1277) # select some x_pixel before summing.
cc0.plot_trace(y_pixel_slice=slice(0, 1), x_pixel_slice=x_slice, as_mean=True)
xlabel("Time in fs")
ylabel("Mean Counts per Peak")
title("Cross-Correlation")
sfg2d.savefig("figures/cc_trace")

cc0.save("combined/pump_probe_iiv")

# Pump SFG

In [ ]:
# Pump-Sfg and Probe-SFG
pump = sfg2d.SfgRecord("").make_avg()
pump_sfg_spec_index = 0 # The y_pixel_index where the pump-sfg data is.
 
pump.base = bg0 # Set correct baseline
pump.norm = norm_spec
pump.sub_base(inplace=True)
pump.metadata["vis_wl"] = vis_wl
pump._wavenumber = None
pump_spectra_slice = slice(pump_sfg_spec_index, pump_sfg_spec_index+1)
pump.save("combined/pump")

In [ ]:
fig, ax = subplots()
pump.plot(
    x_axis="wavenumber",
    y_pixel=pump_spectra_slice,
    
)
xlabel("Wavenumber in 1/cm")
ylabel("Counts in {0}s".format(pump.metadata.get('exposure_time').seconds))
#xlim(2300, 2600)
title("Pump SFG")
sfg2d.savefig("figures/pump")


fig, ax = subplots()
pump.plot_trace(y_pixel_slice=pump_spectra_slice, as_mean=True )
xlabel("Time in fs")
ylabel("Mean Counts")
title("Trace of Pump SFG Signal")
#xlim(-5000, 5000)
sfg2d.savefig("figures/pump_trace")

# Bleach

In [ ]:
g0 = sfg2d.SfgRecord("./").make_avg() # Fill in path


g0.base = bg0 # Use correct baseline
g0.norm = norm_spec
g0.sub_base(inplace=True)
g0.metadata["vis_wl"] = vis_wl
g0._wavenumber = None
g0._pumped_index = 2
g0._unpumped_index = 1
g0.save("combined/gold")

In [ ]:
fig, ax = subplots()
i = 1 # index of the example pump_probe delay 
g0.plot_bleach(attribute='bleach_rel', pp_delays=slice(i, i+1), x_axis='wavenumber');
xlabel("Wavenumber in 1/cm")
ylabel("Mean Relative Bleach")
title("Gold Relative Bleach {0}fs Delay".format(g0.pp_delays[i]))
sfg2d.savefig("figures/gold_bleach_rel_{0}fs".format(g0.pp_delays[i]))

In [ ]:
fig, ax = subplots()
x_pixel_slice = slice(None, None) # Select some x_pixels before summing the bleach
g0.plot_trace(y_axis="get_trace_bleach", attr="bleach_rel", x_pixel_slice=x_pixel_slice);
xlabel("time in fs")
ylabel("Mean relative bleach")
title("Bleach on Gold")

# Combining Scans

In [ ]:
# Fill in paths to files
imp_data = (
    "./",  
    "./", 
)
sample_list = []
for elm in imp_data:
    sfgrecord = sfg2d.SfgRecord(elm)
    ## Remove unneeded frames like so
    #if elm == "": # enter name of the file here
    #    sfgrecord.rawData = sfgrecord.rawData[:,:3] # The second index is the frame slice
    sample_list.append(sfgrecord)
sample = sfg2d.concatenate_list_of_SfgRecords(sample_lis) # This combined the records into a single record
sample.metadata["pump_freq"] = 2500 # You can set a pump frequency in metadata

# 2D Spectrum from Scan

In [ ]:
# Data for axes
x = sample.pp_delays
y = sample.wavenumber[400:1200] # only use a pixel subset.
z = sfg2d.double_resample(sample.bleach.squeeze()[:, 400:1200], 30, axis=-1) # FFT based filters work better here
y_slice=slice(None) # select subslice to calculate the trace from.
# This slice can be set in plot coordinates as well. So if you are using wavenumbers here, then e.g.
# y_slice = (2300, 2500)

fig = plt.figure(figsize=(10.9, 7.5))
fig, ax, axl, axb = sfg2d.contour(
    x, y, z, 
    y_slice=y_slice,
    N=10, vmin=-0.003, vmax=0.001, show_colorbar=False, fig=fig)
axb.set_xlabel("Time in fs")
axl.set_ylabel("Wavenumber in 1/cm")
ax.set_xlim(-5000, 5000)
plt.tight_layout()

#name = "figures/sample_bleach_pump_%i_interval%i-%i_img"%(sample.metadata["pump_freq"],*y_slice)

#sfg2d.savefig(name, dpi=80)